In [1]:
import psycopg2
import pandas as pd

## Make a connection to the existing database, initiate a cursor instance that launches queries and use that to create a new database.

In [2]:
def create_database():
    
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=root")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    cur.execute("DROP DATABASE craftbeers")
    cur.execute("CREATE DATABASE craftbeers")
    
    conn.close()
    
    conn = psycopg2.connect("host=127.0.0.1 dbname=craftbeers user=postgres password=root")
    cur = conn.cursor()
    
    return cur, conn

## Methods to execute query and commit the same during creating and dropping tables.

In [3]:
def drop_tables(cur, con):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()
    

In [4]:
def create_tables(cur, con):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

```
head() -  prints first 5 rows of the dataset.
```

In [5]:
BeersInfo = pd.read_csv("Data/beers.csv")

In [6]:
BeersInfo.head()

,Unnamed: 0,abv,ibu,beer_id,name,style,brewery_id,ounces
0,0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0


```
Filtering the specific columns that we need to work.
```

In [7]:
BeersInfo_clean = BeersInfo[['abv', 'beer_id', 'name', 'style', 'brewery_id', 'ounces']]
BeersInfo_clean.head()

,abv,beer_id,name,style,brewery_id,ounces
0,0.050,1436,Pub Beer,American Pale Lager,408,12.0
1,0.066,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,0.071,2264,Rise of the Phoenix,American IPA,177,12.0
3,0.090,2263,Sinister,American Double / Imperial IPA,177,12.0
4,0.075,2262,Sex and Candy,American IPA,177,12.0


In [8]:
Breweries = pd.read_csv("Data/breweries.csv")
Breweries.head()

,brewery_id,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA


In [9]:
Transport = pd.read_csv("Data/transport.csv")
Transport.head()

,truck_id,model,city,fuel_type,owner
0,5673,Red Truck,Minneapolis,petrol,NaN
1,3986,Tap Truck,Louisville,diesel,NaN
2,5871,Monarch Distributing,Framingham,petrol,NaN
3,2870,Chicago Beverage,San Diego,ECV,NaN
4,3510,Red Truck,San Francisco,diesel,NaN


In [10]:
Transport.columns

Index(['truck_id', 'model', 'city', 'fuel_type', 'owner'], dtype='object')

```
Removing the unwanted column using drop() method.
axis = 1 represents column, 0 represents row
```

In [11]:
Transport = Transport.drop(['owner'], axis=1)
Transport.columns
Transport.head()

,truck_id,model,city,fuel_type
0,5673,Red Truck,Minneapolis,petrol
1,3986,Tap Truck,Louisville,diesel
2,5871,Monarch Distributing,Framingham,petrol
3,2870,Chicago Beverage,San Diego,ECV
4,3510,Red Truck,San Francisco,diesel


In [12]:
cur, conn = create_database()

### Create tables as per the data model - [BeerCraft.drawio]()

In [13]:
breweries_create_table = ("""CREATE TABLE IF NOT EXISTS breweries(
brewery_id INT PRIMARY KEY,
name VARCHAR,
city VARCHAR,
state VARCHAR
)""")

cur.execute(breweries_create_table)
conn.commit()

In [14]:
beers_create_table = ("""CREATE TABLE IF NOT EXISTS beers(
beer_id INT PRIMARY KEY,
abv NUMERIC,
name VARCHAR,
style VARCHAR,
brewery_id INT REFERENCES breweries (brewery_id),
ounces NUMERIC     
)""")

cur.execute(beers_create_table)
conn.commit()

In [15]:
transport_create_table = ("""CREATE TABLE IF NOT EXISTS transport(
truck_id INT PRIMARY KEY,
model VARCHAR,
city VARCHAR,
fuel_type VARCHAR   
)""")

cur.execute(transport_create_table)
conn.commit()


### Post inserting the data, loop thru the dataset while using .iterrows() method from pandas that returns couple of values -  i & row
#### i - index, row - elements of the dataset(from list of lists)
```
Inside the loop, fire the query thru cursor by sending two parameters.
1. insert-syntax thats assigned to a string variable
2. each element in the dataset by converting it to a list
```

In [16]:
breweries_insert = ("""INSERT INTO breweries(
brewery_id,
name,
city,
state)
VALUES(%s, %s, %s, %s)
""")

for i,row in Breweries.iterrows():
    cur.execute(breweries_insert, list(row))

In [17]:
beers_insert = ("""INSERT INTO beers(
abv,
beer_id,
name,
style,
brewery_id,
ounces)
VALUES(%s, %s, %s, %s, %s, %s)
""")

for i,row in BeersInfo_clean.iterrows():
    cur.execute(beers_insert, list(row))

In [18]:
transport_insert = ("""INSERT INTO transport(
truck_id,
model,
city,
fuel_type)
VALUES(%s, %s, %s, %s)
""")

for i,row in Transport.iterrows():
    cur.execute(transport_insert, list(row))

In [19]:
## Dont forget to commit the actions thru conn instance to reflect the data onto tables.
conn.commit()

In [20]:
conn.close()